In [2]:

import numpy as np

import torch
import torch.nn as n
import torch.nn.functional as F
import transformers
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import TensorDataset,DataLoader
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adesh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:

import tqdm
from matplotlib._path import (affine_transform, count_bboxes_overlapping_bbox,
     update_path_extents)

In [4]:
dataset=pd.read_csv("D:\\finalmanthan1.csv")
dataset.dropna(inplace = True)
df = dataset
dataset

,tweet,hate,userID,likes,comments,views,shares,tags
0,!!! RT @mayasolovely: As a woman you shouldn't...,0,9CAA8AE2,712,183,283,246,23
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,1069B853,856,828,625,1,5
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,F4468A56,59,150,778,45,90
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,978BF9DB,183,752,1473,460,87
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1,2AAE69DE,232,453,438,634,6
...,...,...,...,...,...,...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,BEA9A786,187,282,98,318,18
24779,"you've gone and broke the wrong heart baby, an...",0,C632D164,433,166,383,666,62
24780,young buck wanna eat!!.. dat nigguh like I ain...,0,AFD6B7C7,247,375,1324,97,38
24781,youu got wild bitches tellin you lies,1,6A6D8F91,896,186,376,196,71


In [5]:
print("number of tweets belonging to classes 0,1 and 2")
dataset.groupby('hate')['userID'].nunique()

number of tweets belonging to classes 0,1 and 2


hate
0    20070
1     4713
Name: userID, dtype: int64

In [6]:
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
#test_clean = clean_text(test, "tweet")
train_clean = clean_text(df, "tweet")

In [7]:
train_clean


,tweet,hate,userID,likes,comments,views,shares,tags
0,rt as a woman you shouldnt complain about cl...,0,9CAA8AE2,712,183,283,246,23
1,rt boy dats coldtyga dwn bad for cuffin dat ...,0,1069B853,856,828,625,1,5
2,rt dawg rt you ever fuck a bitch and she st...,0,F4468A56,59,150,778,45,90
3,rt ganderson based she look like a tranny,0,978BF9DB,183,752,1473,460,87
4,rt the shit you hear about me might be true ...,1,2AAE69DE,232,453,438,634,6
...,...,...,...,...,...,...,...,...
24778,yous a muthafin lie 8220 pearls emanuel right ...,0,BEA9A786,187,282,98,318,18
24779,youve gone and broke the wrong heart baby and ...,0,C632D164,433,166,383,666,62
24780,young buck wanna eat dat nigguh like i aint fu...,0,AFD6B7C7,247,375,1324,97,38
24781,youu got wild bitches tellin you lies,1,6A6D8F91,896,186,376,196,71


In [8]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.hate==0]
train_minority = train_clean[train_clean.hate==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['hate'].value_counts()

1    20070
0    20070
Name: hate, dtype: int64

In [9]:
from gensim.models import Word2Vec


In [10]:
model = Word2Vec(train_upsampled["tweet"].values, size=50, window=7, min_count=1, workers=4)

In [11]:
def get_features(tweet):
    features=[]
    for word in tweet:
        features.append(model.wv[word])
    return np.mean(features,0)

In [12]:
train_upsampled["features"]=train_upsampled["tweet"].apply(get_features)

In [13]:
data=[]
for i in train_upsampled["features"].values:
    temp=[]
    for j in i:
        temp.append(j)
    data.append(temp)
data=np.array(data)

In [14]:
from sklearn.preprocessing import label_binarize

Y = label_binarize(train_upsampled["hate"].values, classes=[0, 1])
n_classes = Y.shape[1]


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=39)

In [16]:
print(X_train)
print(y_train)

[[ 0.02971563  0.05264218  0.3497425  ...  0.0072559   0.04053199
   0.26874518]
 [ 0.06412695  0.00622537  0.25073102 ...  0.0155072   0.01759142
   0.08829313]
 [-0.0405525   0.10417752  0.29934654 ... -0.14492427 -0.00711682
   0.17960072]
 ...
 [-0.06975789  0.09242813  0.21472533 ...  0.01395796  0.13640368
  -0.02211014]
 [-0.03280688  0.03741189  0.28354976 ... -0.09000807 -0.03710093
   0.17224933]
 [ 0.03803318  0.03960341  0.3367509  ... -0.02272648  0.01084033
   0.23803243]]
[[0]
 [0]
 [0]
 ...
 [0]
 [1]
 [0]]


In [17]:
model_class = transformers.BertModel
tokenizer_class = transformers.BertTokenizer
pretrained_weights = 'bert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
bert_model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
max_seq = 27
def tokenize_text(df, max_seq):
    return [
        tokenizer.encode(text, add_special_tokens=True)[:max_seq] for text in df["tweet"].values
    ]
def pad_text(tokenized_text, max_seq):
    return np.array([el + [0] * (max_seq - len(el)) for el in tokenized_text])
def tokenize_and_pad_text(df, max_seq):
    tokenized_text = tokenize_text(df, max_seq)
    padded_text = pad_text(tokenized_text, max_seq)
    return torch.tensor(padded_text)
def targets_to_tensor(df, target_columns):
    return torch.tensor(df[target_columns].values, dtype=torch.float32)

In [19]:
max = 0
for i in train_upsampled["tweet"]:
  if len(i) > max:
    max = len(i)
print(max)

505


In [20]:
train_upsampled["tweet"][15]

' bitch nigga miss me with it '

In [21]:
#for i in range(0,len(train_upsampled["tweet"])):
#  train_upsampled["tweet"][i] = " ".join(train_upsampled["tweet"][i])

In [22]:
train_upsampled

,tweet,hate,userID,likes,comments,views,shares,tags,features
19036,ray239 the wicked witch of the east is done n...,1,DDD7D5E6,995,155,531,353,7,"[-0.011196802, 0.083247356, 0.2888572, -0.2067..."
18374,hereyezlow u gotta be a pathetic bitch to rid...,1,EF397864,243,58,1136,30,0,"[0.022644665, 0.06128016, 0.3349919, -0.265314..."
7089,that cracker as big as he looks on tv,1,08186A55,1099,988,193,431,50,"[0.06519051, 0.05081605, 0.29209238, -0.252143..."
21564,the guys at la fitness are such fags 128514128557,1,730C4C62,1175,947,375,492,78,"[-0.25268933, -0.017981874, 0.3040056, -0.0669..."
8379,cant wait rt all niggas wanna do at 45 is pop...,1,7C4C5EC5,350,874,240,371,74,"[-0.040221054, 0.12337199, 0.25538245, -0.2512..."
...,...,...,...,...,...,...,...,...,...
24777,youre such a retard i hope you get type 2 diab...,0,7104A67B,292,914,1418,447,35,"[0.087197505, -0.011839661, 0.36090338, -0.226..."
24778,yous a muthafin lie 8220 pearls emanuel right ...,0,BEA9A786,187,282,98,318,18,"[-0.041917752, -0.0028036877, 0.13118997, -0.1..."
24779,youve gone and broke the wrong heart baby and ...,0,C632D164,433,166,383,666,62,"[-0.015208609, 0.056279372, 0.31304246, -0.354..."
24780,young buck wanna eat dat nigguh like i aint fu...,0,AFD6B7C7,247,375,1324,97,38,"[0.0309142, 0.058232315, 0.365421, -0.17720442..."


In [23]:
indices = tokenize_and_pad_text(train_upsampled, max_seq)

[no,_] = list(indices.shape)

In [ ]:
import numpy as np
stepSize = 32
X = np.empty([0,max_seq,768])

for i in range(0,no,stepSize):
  if no - i <= stepSize:
    n = no
  else:
    n = i+stepSize
  with torch.no_grad():
    x = bert_model(indices[i:n,:])[0].detach().numpy()
  X = np.vstack([X,x])

In [ ]:
label_col = ["hate"]
Y = targets_to_tensor(train_upsampled,label_col)